In [1]:
import requests

In [2]:
# !pip3 install requests

In [3]:
url = 'https://books.toscrape.com/'

res = requests.get(url)

In [7]:
res.content

b'<!DOCTYPE html>\n<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->\n    <head>\n        <title>\n    All products | Books to Scrape - Sandbox\n</title>\n\n        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n        <meta name="created" content="24th Jun 2016 09:29" />\n        <meta name="description" content="" />\n        <meta name="viewport" content="width=device-width" />\n        <meta name="robots" content="NOARCHIVE,NOCACHE" />\n\n        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->\n        <!--[if lt IE 9]>\n        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>\n        <![endif]-->\n\n        \n            <link rel="shortcut icon" hre

In [6]:
from bs4 import BeautifulSoup 

In [8]:
tree = BeautifulSoup(res.content, 'html')

In [12]:
tree.title.text.strip()

'All products | Books to Scrape - Sandbox'

In [74]:
tree.find_all('p', {'class': 'price_color'})

[<p class="price_color">£51.77</p>,
 <p class="price_color">£53.74</p>,
 <p class="price_color">£50.10</p>,
 <p class="price_color">£47.82</p>,
 <p class="price_color">£54.23</p>,
 <p class="price_color">£22.65</p>,
 <p class="price_color">£33.34</p>,
 <p class="price_color">£17.93</p>,
 <p class="price_color">£22.60</p>,
 <p class="price_color">£52.15</p>,
 <p class="price_color">£13.99</p>,
 <p class="price_color">£20.66</p>,
 <p class="price_color">£17.46</p>,
 <p class="price_color">£52.29</p>,
 <p class="price_color">£35.02</p>,
 <p class="price_color">£57.25</p>,
 <p class="price_color">£23.88</p>,
 <p class="price_color">£37.59</p>,
 <p class="price_color">£51.33</p>,
 <p class="price_color">£45.17</p>]

In [79]:
tree.h3.a

<a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a>

In [23]:
prices_dirty = tree.find_all('p', {'class': 'price_color'})

prices = [float(el.text[1:]) for el in prices_dirty]

In [42]:
tree.h3.a.get('title')

'A Light in the Attic'

In [35]:
tree.a

<a href="index.html">Books to Scrape</a>

In [43]:
tree.a.text

'Books to Scrape'

In [44]:
tree.a.get('href')

'index.html'

In [45]:
tree.h3.a

<a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a>

In [49]:
books = tree.find_all('article', {'class': 'product_pod'})

In [83]:
prices = []
titles = []
for book in books:
    price = float(book.find('p', {'class': 'price_color'}).text[1:])
    title = book.h3.a.get('title')
    
    prices.append(price)
    titles.append(title)

In [88]:
d = {'title': titles, 
    'price': prices}

In [90]:
import pandas as pd

In [91]:
df= pd.DataFrame(d)

In [92]:
df

,title,price
0,A Light in the Attic,51.77
1,Tipping the Velvet,53.74
2,Soumission,50.10
3,Sharp Objects,47.82
4,Sapiens: A Brief History of Humankind,54.23
5,The Requiem Red,22.65
6,The Dirty Little Secrets of Getting Your Dream...,33.34
7,The Coming Woman: A Novel Based on the Life of...,17.93
8,The Boys in the Boat: Nine Americans and Their...,22.60
9,The Black Maria,52.15


### Делаем красиво

In [115]:
def get_books_info(url):
    res = requests.get(url)
    tree = BeautifulSoup(res.content, 'html')
    
    books = tree.find_all('article', {'class': 'product_pod'})
    prices = []
    titles = []
    hrefs = []
    for book in books:
        price = float(book.find('p', {'class': 'price_color'}).text[1:])
        title = book.h3.a.get('title')
        href = 'https://books.toscrape.com/catalogue/' + book.h3.a.get('href')
        
        
        hrefs.append(href)
        prices.append(price)
        titles.append(title)
        
    return prices, titles, hrefs

In [116]:
prices, titles, hrefs = get_books_info('https://books.toscrape.com/catalogue/page-1.html')

In [120]:
res = requests.get(hrefs[0])
tree = BeautifulSoup(res.content, 'html')

In [122]:
import time

In [124]:
prices_all = []
titles_all = []
hrefs_all = []
for p in range(1, 51):
    url = f'https://books.toscrape.com/catalogue/page-{p}.html'
    prices, titles, hrefs = get_books_info(url)
    
    prices_all.extend(prices)
    titles_all.extend(titles)
    hrefs_all.extend(hrefs)
    
    time.sleep(1)

In [101]:
d = {'title': titles_all, 
    'price': prices_all}

df = pd.DataFrame(d)

In [109]:
df[df.price > 50]

,title,price
0,A Light in the Attic,51.77
1,Tipping the Velvet,53.74
2,Soumission,50.10
4,Sapiens: A Brief History of Humankind,54.23
9,The Black Maria,52.15
...,...,...
978,"Giant Days, Vol. 1 (Giant Days #1-4)",56.76
984,"Eat, Pray, Love",51.32
995,Alice in Wonderland (Alice's Adventures in Won...,55.53
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",57.06
